In [ ]:

from keras.models import load_model
from numpy.random import randint
import cv2
import numpy as np
import matplotlib.pyplot as plt



generator = load_model('gen_e_10.h5', compile=False)

ct_lr = cv2.imread("CAP_SCD0000101_MR__hrt_raw_20120813120600157_1.jpg")
ct_hr = cv2.imread("CAP_SCD0000101_MR__hrt_raw_20120813120600157_1 HR.jpg")

ct_lr = cv2.cvtColor(ct_lr, cv2.COLOR_BGR2RGB)
ct_hr = cv2.cvtColor(ct_hr, cv2.COLOR_BGR2RGB)

ct_lr = ct_lr / 255.
ct_hr = ct_hr / 255.

ct_lr = np.expand_dims(ct_lr, axis=0)
ct_hr = np.expand_dims(ct_hr, axis=0)

generated_ct_hr = generator.predict(ct_lr)

# plot all three images
plt.figure(figsize=(16, 8))
plt.subplot(231)
plt.title('LR Image')
plt.imshow(ct_lr[0,:,:,:])
plt.subplot(232)
plt.title('Superresolution')
plt.imshow(generated_ct_hr[0,:,:,:])
plt.subplot(233)
plt.title('Orig. HR image')
plt.imshow(ct_hr[0,:,:,:])

plt.show()

In [ ]:
import tensorflow as tf
from tensorflow.keras.losses import mse
from tensorflow.image import ssim
import matplotlib.pyplot as plt

# Convert the images to tensors
generated_ct_hr_tensor = tf.convert_to_tensor(generated_ct_hr, dtype=tf.float32)
ct_hr_tensor = tf.convert_to_tensor(ct_hr, dtype=tf.float32)
print("generated_ct_hr_tensor",generated_ct_hr_tensor.shape)

# Calculate the PSNR  # PSNR = 10 * log10((MAX^2) / MSE)
mse_value = mse(ct_hr_tensor, generated_ct_hr_tensor)
psnr = tf.squeeze(tf.image.psnr(ct_hr_tensor, generated_ct_hr_tensor, max_val=1.1))  # max_val should be set to the maximum pixel value of the image
# Calculate the SSIM
ssim_value = ssim(ct_hr_tensor, generated_ct_hr_tensor, max_val=1.0) #SSIM(x,y) = (l(x,y) * c(x,y) * s(x,y))


# Print the PSNR and SSIM values
psnr_title = f"PSNR: {psnr.numpy():.2f} dB"
ssim_title = f"SSIM: {ssim_value.numpy().item():.4f}"

# Plot the images with titles
plt.figure(figsize=(16, 8))
plt.subplot(231)
plt.title('LR Image')
plt.imshow(ct_lr[0,:,:,:])
plt.subplot(232)
plt.title('Super resolution\n' + ssim_title + '\n' + psnr_title)
plt.imshow(generated_ct_hr[0,:,:,:])
plt.subplot(233)
plt.title('Orig. HR image')
plt.imshow(ct_hr[0,:,:,:])

plt.show()